In [1]:
!conda install torchvision --yes
!conda install torch --yes

Retrieving notices: ...working... done
Channels:
 - defaults
Platform: osx-64
Solving environment: done

# All requested packages already installed.

Channels:
 - defaults
Platform: osx-64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - torch

Current channels:

  - defaults

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms 
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
import sys
print(sys.version)

3.8.18 (default, Sep 11 2023, 08:17:33) 
[Clang 14.0.6 ]


In [4]:
# Defining hyperparameters
input_size = 28  # Total size of the image 28*28
num_classes = 10  # Number of types of labels
num_epochs = 3  # Total cycle time of training
batch_size = 64  # One handful (batch) size, 64 images

# Train set
train_dataset = datasets.MNIST(root='./data',  
                            train=True,   
                            transform=transforms.ToTensor(),  
                            download=True) 

# Test set
test_dataset = datasets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Building batch data
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

### Convolutional network module construction
- Generally, the convolution, relu and pooling layers can be written as a package.
- Note that the final result of convolution is still a feature map, which needs to be converted into a vector to do classification or regression tasks.

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(  # Input size (1, 28, 28)
            nn.Conv2d(
                in_channels=1,    # grayscale image
                out_channels=16,  # How many feature maps to get
                kernel_size=5,    # Convolution kernel size
                stride=1,         # footsteps
                padding=2         # If you want the convolution to be the same size as the original, you need to set padding=(kernel_size-1)/2 if stride=1
            ),
            nn.ReLU(),            # Relu layer
            nn.MaxPool2d(kernel_size=2), # Perform pooling operation (2x2 region), the output is: (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # Input for the next package (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),     # Output (32, 14, 14)
            nn.ReLU(),                      # Relu layer
            nn.Conv2d(32, 32, 5, 1, 2),
            nn.ReLU(),
            nn.MaxPool2d(2),                # Output (32, 7, 7)
        )
        self.conv3 = nn.Sequential(         # Input for the next package (16, 14, 14)
            nn.Conv2d(32, 64, 5, 1, 2),     #  (32, 14, 14)
            nn.ReLU(),                      # Output (64, 7, 7)
        )
        
        self.out = nn.Linear(64 * 7 * 7, 10)   # Results obtained from the fully connected layer
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)           # flatten operation resulting in: (batch_size, 32 * 7 * 7)
        output = self.out(x)
        return output
        

### Accuracy as an assessment criterion

In [6]:
def accuracy(predictions, labels):
    pred = torch.max(predictions.data, 1)[1] 
    rights = pred.eq(labels.data.view_as(pred)).sum() 
    return rights, len(labels) 

### Train the network model

In [7]:
# Instantiated
net = CNN() 
# Loss function
criterion = nn.CrossEntropyLoss() 
# Optimizer
optimizer = optim.Adam(net.parameters(), lr=0.001) # Define optimizer, ordinary stochastic gradient descent algorithm

# Start training cycle
for epoch in range(num_epochs):
    # The result of the current epoch is saved
    train_rights = [] 
    
    for batch_idx, (data, target) in enumerate(train_loader):  # Loop for each batch in the container
        net.train()                             
        output = net(data) # Predicted value
        loss = criterion(output, target) # Loss value
        optimizer.zero_grad() # Gradient clearing
        loss.backward() # Backward propagation
        optimizer.step() # Parameter update
        right = accuracy(output, target) # How many get right
        train_rights.append(right) # Save the right

    
        if batch_idx % 100 == 0: 
            
            net.eval() # Verification mode
            val_rights = [] 
            
            for (data, target) in test_loader:
                output = net(data) 
                right = accuracy(output, target) 
                val_rights.append(right)
                
            # Calculation of accuracy
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('Current epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}\tTrain set accuracy: {:.2f}%\tTest Set accuracy: {:.2f}%'.format(
                epoch, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.data, 
                100. * train_r[0].numpy() / train_r[1], 
                100. * val_r[0].numpy() / val_r[1]))

Current epoch: 0 [0/60000 (0%)]	 Loss: 2.302157	Train set accuracy: 10.94%	Test Set accuracy: 9.74%
Current epoch: 0 [6400/60000 (11%)]	 Loss: 0.314603	Train set accuracy: 78.17%	Test Set accuracy: 95.29%
Current epoch: 0 [12800/60000 (21%)]	 Loss: 0.091689	Train set accuracy: 86.64%	Test Set accuracy: 96.14%
Current epoch: 0 [19200/60000 (32%)]	 Loss: 0.203928	Train set accuracy: 90.02%	Test Set accuracy: 96.70%
Current epoch: 0 [25600/60000 (43%)]	 Loss: 0.124109	Train set accuracy: 91.77%	Test Set accuracy: 97.82%
Current epoch: 0 [32000/60000 (53%)]	 Loss: 0.062710	Train set accuracy: 92.95%	Test Set accuracy: 98.06%
Current epoch: 0 [38400/60000 (64%)]	 Loss: 0.002474	Train set accuracy: 93.72%	Test Set accuracy: 98.39%
Current epoch: 0 [44800/60000 (75%)]	 Loss: 0.066669	Train set accuracy: 94.36%	Test Set accuracy: 98.93%
Current epoch: 0 [51200/60000 (85%)]	 Loss: 0.072030	Train set accuracy: 94.83%	Test Set accuracy: 98.38%
Current epoch: 0 [57600/60000 (96%)]	 Loss: 0.121598	